In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
from shapely.geometry import Point
from sklearn import preprocessing

In [2]:
trees = pd.read_excel('../Data/Raw/Treetracker Site Table.xlsx')
owners = gpd.read_file('../Data/Raw/Master_Building/CH_Only_Clip.shp')

In [3]:
trees = trees[['Address', 'Diameter', 'Condition Wood', 'Condition Leaves', 'X_Coord', 'Y_Coord']]
trees.dropna(inplace=True)

In [4]:
conditions = {'Dead': 0, 'Poor': 1, 'Fair': 2, 'Good': 3, 'Excellent': 4}
trees['Condition Wood'] = trees['Condition Wood'].map(conditions)
trees['Condition Leaves'] = trees['Condition Leaves'].map(conditions)
min_max_scaler = preprocessing.MinMaxScaler((0,4))
trees['Diameter'] = min_max_scaler.fit_transform(np.array(trees['Diameter'].astype(float)).reshape(-1,1))
trees['Tree_Score'] = trees['Diameter'] + trees['Condition Wood'] + trees['Condition Leaves']

In [5]:
tree_scores = trees[['Address', 'Tree_Score']].groupby('Address').mean()

In [6]:
tree_scores.to_csv('../Data/processed/tree_scores.csv')

In [7]:
def fix_address(row):
    num = row['Address'].split(' ')[2]
    st = row['Address'].split(' ')[0] + ' ' + row['Address'].split(' ')[1]
    add = num + ' ' + st
    return add

In [8]:
tree_scores.reset_index(inplace=True)
tree_scores['Address'] = tree_scores.apply(fix_address, axis=1)
tree_scores['Address'] = map(lambda x: x.upper(), tree_scores['Address'])

In [9]:
owners = owners[['parcel_add', 'geometry']]
owners.dropna(inplace=True)

In [10]:
def fix_owners_add(row):
    return row['parcel_add'].split(',')[0]
owners['parcel_add'] = owners.apply(fix_owners_add, axis=1)

In [11]:
tree_scores_shp = pd.merge(tree_scores, owners, how='inner', left_on='Address', right_on='parcel_add')

In [12]:
tree_scores_shp = gpd.GeoDataFrame(tree_scores_shp, crs = {'init' :'epsg:4326'})

In [13]:
tree_scores_shp.to_file('../Data/processed/shapefiles/tree_scores.shp')